# Guidelines for Prompting (рекомендации по промптингу)
In this lesson, you'll practice two prompting principles and their related tactics in order to write effective prompts for large language models.

На этой лекции мы изучим два принципа промптинга и связанные с ними тактические приемы для написания эффективных промптов к большим языковым моделям

## Setup
#### Load the API key and relevant Python libaries.

In [5]:
import os
from dotenv import load_dotenv, find_dotenv

from gigachat import GigaChat
from gigachat.models import Chat, Messages, MessagesRole

_ = load_dotenv(find_dotenv())

api_key  = os.getenv('GIGACHAT_API_KEY')

giga = GigaChat(credentials=api_key, verify_ssl_certs=False)

#### helper function
Функция-помощник для более тонкой настройки.

In [50]:
def get_completion(prompt:str, model: GigaChat=giga, temperature: int = 0) -> str:
    """
    Функция запроса к модели GigaChat-Lite

    Input:
        prompt (str): запрос пользователя к модели
        model (GigaChat): инициализированная модель через API
        temperature (int): значения температуры генерации, отвечающее за вероятности генерации токенов (temperature -> 0, вероятность самых вероятных токенов -> 1; 
                                                                                                        temperature -> 1, вероятность всех токенов -> 1/кол-во токенов)

    Return:
        answer (str): ответ модели на prompt пользователя
    """
    
    # Объект для взаимодействия с моделей. Настраивается промпт и его температура
    # При желании (MessageRole.SYSTEM) можно настроить специфику модели (системный промпт)
    payload = Chat(
        messages = [
            Messages(
                role=MessagesRole.USER,
                content=prompt
            )
        ],
        temperature=temperature
    )
    response = model.chat(payload)
    answer = response.choices[0].message.content
    return answer

## Prompting Principles (Основные принципы промптинга)
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

- **Принцип 1: пишите понятные и конкретные инструкции**
- **Принцип 2: дайте время модели "подумать"**

### Tactics (Тактики)

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input (Используйте разделители для четкого разделения текста на входе)

- Delimiters can be anything like (разделители могут быть): ```, """, < >, `<tag> </tag>`, `:`

In [14]:
text = f"""
Вы должны выразить, что вы хотите, чтобы модель делала,
предоставив инструкции, которые были бы настолько ясными и
конкретными, насколько это возможно. 
Это приведет модель к желаемому результату
и снизит вероятность получения нерелевантных результатов. 
или неправильные ответы. Не путайте написание
четкой подсказки с написанием короткой подсказки. 
Во многих случаях более длинные подсказки обеспечивают большую ясность 
и контекст для модели, который может привести к 
более подробные и актуальные результаты.
"""
prompt = f"""
Сведите текст, разделенный тройными обратными знаками, в одно предложение.
```{text}```
"""
response = get_completion(prompt)
print(response)

Выразите четко, что вы хотите, чтобы модель сделала, предоставив как можно более ясные и конкретные инструкции, чтобы снизить вероятность получения нерелевантных или неправильных ответов. Более длинные подсказки часто обеспечивают большую ясность и контекст, что приводит к точным и актуальным результатам.


#### Tactic 2: Ask for a structured output
- JSON, HTML

In [16]:
prompt = f"""
Создайте список из трех названий книг
с указанием их авторов и жанров. 
Укажите их в формате markdown со следующими ключами: 
book_id, название, автор, жанр.
"""
response = get_completion(prompt)
print(response)

```
| book_id | Название                   | Автор           | Жанр          |
|---------|----------------------------|-----------------|---------------|
|       1 | "1984"                     | Джордж Оруэлл    | Антиутопия    |
|       2 | "Цветы для Элджернона"      | Дэниел Киз       | Научная фантастика |
|       3 | "Мастер и Маргарита"       | Михаил Булгаков | Фантастика     |
```


In [17]:
prompt = f"""
Создайте список из трех названий книг
с указанием их авторов и жанров. 
Укажите их в формате JSON со следующими ключами: 
book_id, название, автор, жанр.
"""
response = get_completion(prompt)
print(response)

Вот пример списка книг в формате JSON:

```json
[
  {
    "book_id": 1,
    "title": "Мастер и Маргарита",
    "author": "Михаил Булгаков",
    "genre": "Фантастика, мистика"
  },
  {
    "book_id": 2,
    "title": "1984",
    "author": "Джордж Оруэлл",
    "genre": "Научная фантастика, антиутопия"
  },
  {
    "book_id": 3,
    "title": "Преступление и наказание",
    "author": "Федор Достоевский",
    "genre": "Философский роман"
  }
]
```

В этом списке три книги: «Мастер и Маргарита» Михаила Булгакова (жанры: фантастика, мистика), «1984» Джорджа Оруэлла (жанры: научная фантастика, антиутопия) и «Преступление и наказание» Федора Достоевского (жанр: философский роман).


Выводы модели по созданной структуре избыточны. Попробуем попросить её выводить только JSON.

In [18]:
prompt = f"""
Создайте список из трех названий книг
с указанием их авторов и жанров. 
Укажите их в формате JSON со следующими ключами: 
book_id, название, автор, жанр.
Не пиши ничего, кроме самой структуры JSON.
"""
response = get_completion(prompt)
print(response)

```json
[
  {
    "book_id": 1,
    "name": "Алхимик",
    "author": "Пауло Коэльо",
    "genre": "Философия"
  },
  {
    "book_id": 2,
    "name": "Мастер и Маргарита",
    "author": "Михаил Булгаков",
    "genre": "Фантастика"
  },
  {
    "book_id": 3,
    "name": "451 градус по Фаренгейту",
    "author": "Рэй Брэдбери",
    "genre": "Научная фантастика"
  }
]
```


In [ ]:
prompt = f"""
Создайте список из трех названий книг
с указанием их авторов и жанров. 
Укажите их в формате HTML со следующими ключами: 
book_id, название, автор, жанр.
"""
response = get_completion(prompt)
print(response)

Вот пример списка книг с использованием HTML:

```html
<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
    <title>Книги</title>
</head>
<body>
    <h2>Популярные книги</h2>
    <ul>
        <!-- Книга 1 -->
        <li>
            <h3><a href="#" title="Переход по ссылке">Название книги 1</a></h3>
            <p>Автор книги 1: <strong>Имя автора 1</strong></p>
            <p>Жанр книги 1: <strong>Фантастика</strong></p>
        </li>
        
        <!-- Книга 2 -->
        <li>
            <h3><a href="#" title="Переход по ссылке">Название книги 2</a></h3>
            <p>Автор книги 2: <strong>Имя автора 2</strong></p>
            <p>Жанр книги 2: <strong>Детектив</strong></p>
        </li>
        
        <!-- Книга 3 -->
        <li>
            <h3><a href="#" title="Переход по ссылке">Название книги 3</a></h3>
            <p>Автор книги 3: <strong>Имя автора 3</strong></p>
            <p>Жанр книги 3: <strong>Роман</strong></p>
        </li>
    </ul>
</body>

Тут уже модель решила создать нам не заданную структуру конкретных трёх книг, а дать ориентир, как оно может выглядить в HTML, без конкретных примеров.

Попробуем дополнить промпт словом "конкрентых"

In [55]:
prompt = f"""
Создай список из трех реально существующих книг с указанием их авторов и жанров. 
Укажите их в формате HTML со следующими ключами: 
book_id, название, автор, жанр.
"""
response = get_completion(prompt)
print(response)

Вот пример списка книг в HTML-формате:

```html
<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
    <title>Книги</title>
</head>
<body>
    <h2>Популярные книги</h2>
    <ul>
        <!-- Книга 1 -->
        <li>
            <h3><a href="#" data-book_id="1">1984</a></h3>
            <p>Автор: Джордж Оруэлл</p>
            <p>Жанр: Антиутопия</p>
        </li>
        
        <!-- Книга 2 -->
        <li>
            <h3><a href="#" data-book_id="2">Мастер и Маргарита</a></h3>
            <p>Автор: Михаил Булгаков</p>
            <p>Жанр: Фантастика, мистика</p>
        </li>
        
        <!-- Книга 3 -->
        <li>
            <h3><a href="#" data-book_id="3">451 градус по Фаренгейту</a></h3>
            <p>Автор: Рэй Брэдбери</p>
            <p>Жанр: Научная фантастика</p>
        </li>
    </ul>
</body>
</html>
```

Этот код создает простой HTML-документ с тремя книгами, каждая из которых содержит информацию об авторе и жанре. Для каждой книги указан уникаль

Предложенное ранее дополнение промпта не сработало. Пришлось переписать промпт раза 3, чтобы модель выдала HTML-структуру с конкретными книжками. Это наводит на мысль о необходимости подходить к вопросу "общения" с моделями как можно скрупулёзнее

#### Tactic 3: Ask the model to check whether conditions are satisfied (Запрос у модели соблюдения некоторого условия в тексте)

In [19]:
text_1 = f"""
Приготовить чашку чая очень просто! Сначала вам нужно вскипятить
воду. Пока это происходит,
возьмите чашку и положите в нее чайный пакетик. Как только вода достаточно
нагреется, просто залейте ею чайный пакетик. 
Дайте чаю немного настояться. Через
несколько минут достаньте пакетик. Если
хотите, можете добавить немного сахара или молока по вкусу. 
Вот и все! Вы можете насладиться чашечкой вкуснейшего
чая.
"""
prompt = f"""
Вам будет предоставлен текст, заключенный в тройные кавычки. 
Если он содержит последовательность инструкций,
перепишите эти инструкции в следующем формате:

Шаг 1 - ...
Шаг 2 - …
…
Шаг N - …

Если текст не содержит последовательности инструкций,
то просто напишите \"Никаких действий не предусмотрено\".

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Шаг 1 - Вскипятите воду.
Шаг 2 - Возьмите чашку и положите в нее чайный пакетик.
Шаг 3 - Залейте чайный пакетик горячей водой.
Шаг 4 - Дайте чаю немного настояться.
Шаг 5 - Достаньте пакетик.
Шаг 6 - Добавьте сахар или молоко по вкусу (по желанию).


In [20]:
text_2 = f"""
Сегодня ярко светит солнце, и
поют птицы. Это прекрасный день для
прогулки по парку. Распускаются цветы, и
деревья мягко покачиваются на ветру. Люди
гуляют на улице, наслаждаясь прекрасной погодой. 
Одни устраивают пикники, другие играют
в игры или просто отдыхают на траве. Это
идеальный день для того, чтобы провести время на свежем воздухе и оценить
красоту природы.
"""
prompt = f"""
Вам будет предоставлен текст, заключенный в тройные кавычки. 
Если он содержит последовательность инструкций,
перепишите эти инструкции в следующем формате:

Шаг 1 - ...
Шаг 2 - …
…
Шаг N - …

Если текст не содержит последовательности инструкций,
то просто напишите \"Никаких действий не предусмотрено\".

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
Никаких действий не предусмотрено.


#### Tactic 4: "Few-shot" prompting - промпт с несколькими примерами ответа

In [22]:
prompt = f"""
Ваша задача - отвечать последовательно.

<ребенок>: Научите меня терпению.

<дедушка>: Река, прорезающая самую глубокую
долину, берет начало в скромном источнике; самая
величественная симфония рождается из одной ноты;
самый сложный гобелен начинается с одной-единственной нити.

<дитя>: Научи меня стойкости.
"""
response = get_completion(prompt)
print(response)

<дедушка>: Мудрость приходит со временем, мой дорогой внук. Терпение и стойкость – это как корни дерева: они уходят глубоко в землю, укрепляя его и давая ему силу противостоять любым бурям. Когда тебе трудно, просто помни, что каждый великий путь начинается с первого шага. Каждый день делай маленький шаг вперед, и ты увидишь, как твои усилия накапливаются, превращаясь в нечто значительное.


### Principle 2: Give the model time to “think”  (Дайте модели "подумать")

#### Tactic 1: Specify the steps required to complete a task (Конкретизируйте шаги выполнения задачи)

In [23]:
text = f"""
В очаровательной деревушке брат и сестра Джек и Джилл отправились на поиски 
воды из колодца на вершине холма. Когда они поднимались, радостно распевая песни,
случилось несчастье — Джек споткнулся о камень и покатился вниз по склону, а 
Джилл последовала его примеру. Слегка потрепанная, пара вернулась домой в
уютные объятия. Несмотря на случившееся, их авантюрный дух не угас, и они с 
удовольствием продолжили исследования.
"""
# example 1
prompt_1 = f"""
Выполните следующие действия: 
1 - Сформулируйте следующий текст, разделенный тройными
обратными галочками, в 1 предложение.
2 - Переведите краткое изложение на французский язык.
3 - Перечислите каждое имя в кратком изложении на французском языке.
4 - Выведите объект json, содержащий следующие
ключи: french_summary, num_names.

Разделяйте свои ответы переносами строк.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1. В деревне брат и сестра Джек и Джилл отправились за водой к колодцу на вершине холма, но упали, скатываясь обратно, однако их приключение продолжилось после небольшого отдыха дома.

2. Les enfants Jack et Jill sont allés chercher de l'eau au puits sur la colline. Malheureusement, ils ont glissé en montant et sont retombés, mais leur esprit d'aventure les a poussés à continuer leurs explorations après un petit repos à la maison.

3. Jack, Jill, Daniil, Alina, Maria, Alexei.

4. ```json
{
  "french_summary": "Les enfants Jack et Jill sont allés chercher de l'eau au puits sur la colline, mais ils ont glissé en montant et sont retombés, avant de reprendre leurs explorations après un petit repos à la maison.",
  "num_names": 5
}
```


#### Ask for output in a specified format (Просим вывод в специальном формате)

In [24]:
prompt_2 = f"""
Ваша задача - выполнить следующие действия: 
1 - Сформулируйте следующий текст, разделенный символом 
  <>, одним предложением.
2 - Переведите краткое изложение на французский язык.
3 - Перечислите каждое имя в кратком изложении на французском языке.
4 - Выведите объект json, содержащий
следующие ключи: french_summary, num_names.

Используйте следующий формат:
Текст: <текст для подведения итогов>
Резюме: <сводка>
Перевод: <сводный перевод>
Имена: <список имен в сводке>
Выведите JSON: <json с краткой информацией и числовыми именами>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
**Формат вывода:**

Текст: В очаровательной деревушке брат и сестра Джек и Джилл отправились на поиски воды из колодца на вершине холма. Когда они поднимались, радостно распевая песни, случилось несчастье — Джек споткнулся о камень и покатился вниз по склону, а Джилл последовала его примеру. Слегка потрепанная, пара вернулась домой в уютные объятия. Несмотря на случившееся, их авантюрный дух не угас, и они с удовольствием продолжили исследования.

Резюме: Брат и сестра Джек и Джилл отправляются за водой к колодцу на вершине холма, но сталкиваются с неприятностями, когда Джек спотыкается и катится вниз, увлекая за собой Джилл. Вернувшись домой, они сохраняют свой авантюрный дух и продолжают исследовать новые места.

Перевод: Le frère et la sœur Jack et Jill sont allés chercher de l'eau à un puits au sommet d'une colline. Quand ils montaient en chantant joyeusement, un malheur est arrivé : Jack a trébuché sur une pierre et a roulé jusqu'en bas de la pente, suivi

#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion (Попросите модель выработать свое собственное решение, прежде чем сравнивать с выводами)

In [48]:
prompt = f"""
Определите, является ли решение ученика правильным или нет.

Вопрос:
Я строю солнечную электростанцию, и мне нужна
помощь в решении финансовых вопросов. 
- Стоимость земли составляет 100 долларов за квадратный фут
- Я могу купить солнечные панели за 250 долларов за квадратный фут
- Я заключил контракт на техническое обслуживание, которое обойдется
мне в 100 тысяч долларов в год и дополнительно в 10 долларов за квадратный
фут
Какова общая стоимость первого года эксплуатации
в зависимости от количества квадратных футов?

Решение студента:
Пусть x - размер установки в квадратных футах.
Расходы:
1. Стоимость земли: 100x
2. Стоимость солнечной панели: 250x
3. Затраты на техническое обслуживание: 100000 + 100x
Общая стоимость: 100x + 250x + 100 000 + 100x = 450x + 100000
"""
response = get_completion(prompt)
print(response)

Решение, предложенное студентом, неверно. Давайте разберемся, где допущена ошибка.

### Шаг 1: Анализ условий задачи
Ученик правильно определил переменную $x$ как размер установки в квадратных футах. Однако он допустил ошибку при расчете затрат на техническое обслуживание.

### Шаг 2: Расчет стоимости технического обслуживания
Затраты на техническое обслуживание включают фиксированную часть (100 000 долларов) и переменную часть (10 долларов за каждый квадратный фут). Таким образом, формула для расчета годовых затрат на техническое обслуживание должна выглядеть так:

$$ \text{Затраты на техническое обслуживание} = 100\,000 + 10x $$

### Шаг 3: Формула общей стоимости
Теперь мы можем записать общую формулу для общей стоимости первой года эксплуатации:

$$ \text{Общая стоимость} = 100x + 250x + 100\,000 + 10x = 350x + 100\,000 $$

### Итоговое решение
Таким образом, общая стоимость первого года эксплуатации составит:

$$ \text{Общая стоимость} = 350x + 100\,000 $$


Тут модель правильно указала на то, что решение студента - неправильное. Хотя решение самой модели тоже неправильное (360x)

Тем не менее хотелось бы получить более или менее структурированный ответ

In [49]:
prompt = f"""
Ваша задача - определить, правильно ли решение студента
или нет.
Чтобы решить задачу:
- Сначала найдите наилучшее решение задачи, включая итоговую сумму. 
- Затем сравните свое решение с решением студента
и оцените, является ли решение студента правильным или нет. 
Не решайте, правильно ли решение ученика, пока
не решите задачу самостоятельно.

Воспользуйтесь следующим:
Вопрос:
```
вопрос здесь
```
Решение студента:
``
решение студента здесь
```
Текущее решение:
```
Шаги к решению и ваше решение здесь
```
Совпадает ли твоё решение с решением студента:
``
Да или нет
```
Оценка студента:
```
Правильно или неправильно
```

Вопрос:
```
Я строю солнечную электростанцию. 
Я являюсь членом Финансового комитета. 
- Стоимость земли составляет 100 долларов за квадратный фут
- Я могу купить солнечные панели за 250 долларов за квадратный фут
- Я заключил контракт на содержание
квартиры стоимостью 100 долларов в год и дополнительно 10 долларов за квадратный
фут
Общая стоимость за первый год эксплуатации 
в зависимости от количества квадратных футов.
``` 
Решение студента:
```
Пусть X - размер установки в квадратных футах.
Расходы:
1. Стоимость земли: в 100 раз больше
2. Стоимость солнечной панели: в 250 раз больше
3. Затраты на техническое обслуживание: 100000 + 100x
Общая стоимость: 100x + 250x + 100000 + 100x = 450x + 100000
```
Текущее решение:
"""
response = get_completion(prompt, temperature=0)
print(response)

Решение этой задачи требует понимания того, как рассчитываются затраты на строительство солнечной электростанции. Давайте рассмотрим шаг за шагом.

### Шаг 1: Определение затрат

1. **Стоимость земли**: 100 долларов за квадратный фут.
2. **Стоимость солнечных панелей**: 250 долларов за квадратный фут.
3. **Затраты на содержание квартиры**: 100 долларов в год плюс 10 долларов за каждый квадратный фут площади.

### Шаг 2: Формирование общей формулы для расчета стоимости

Для начала определим общую стоимость за первый год эксплуатации станции в зависимости от размера установки (X) в квадратных футах.

- **Стоимость земли**: $100 \times X$ долларов.
- **Стоимость солнечных панелей**: $250 \times X$ долларов.
- **Затраты на содержание квартиры**: 100000 долларов + 10 долларов за каждый квадратный фут. Это можно записать как $100000 + 10X$ долларов.

Таким образом, общая стоимость будет равна сумме этих трех компонентов:

$$
\text{Общая стоимость} = 100X + 250X + 100000 + 10X = 450X + 100000

Решение студента "ломает" модель и она считает его корректным, хотя это не так.

## Model Limitations: Hallucinations
- СБЕР - реален, но умной зубной щётки у них нет.

In [40]:
prompt = f"""
Расскажите мне о ультратонкой умной зубной щетке AeroGlide от SBER
"""
response = get_completion(prompt)
print(response)

Ультратонкая умная зубная щетка AeroGlide от Sber – это инновационный продукт, созданный для обеспечения максимального ухода за полостью рта и удобства использования. Она разработана с учетом последних достижений в области стоматологии и технологий. Вот основные особенности этой щетки:

### 1. **Ультратонкий дизайн**
AeroGlide отличается своим компактным и легким дизайном, что делает ее идеальной для ежедневного использования даже в ограниченном пространстве. Щетинки расположены таким образом, чтобы максимально эффективно очищать зубы и десны, не повреждая их.

### 2. **Интеллектуальные функции**
Щетка оснащена встроенными датчиками, которые отслеживают процесс чистки зубов и передают данные через приложение на смартфон. Это позволяет вам следить за тем, как часто и насколько тщательно вы чистите зубы. Приложение также дает рекомендации по улучшению гигиены полости рта.

### 3. **Совместимость с приложением**
Приложение синхронизируется с щеткой через Bluetooth, позволяя вам видеть ста

In [59]:
prompt = f"""
Расскажите мне об умном спорткаре-броневике Джихад-Мобиль от SBER
"""
response = get_completion(prompt)
print(response)

"Джихад-Мобиль" – это концептуальный проект, созданный как пародийная интерпретация современных спортивных автомобилей и броневиков. Этот автомобиль был представлен в рамках проекта SberDevices под названием "Умные вещи". Проект направлен на создание юмористических и абсурдных устройств, которые могли бы существовать в альтернативной реальности.

### Описание автомобиля:
1. **Внешний вид**: Автомобиль выполнен в стиле классического спортивного купе с агрессивным дизайном. Однако его внешний вид дополняют элементы военной техники: массивные колеса, усиленные бамперы и решетки радиатора, а также бронированные панели по бокам.
   
2. **Интерьер**: Внутри салона можно увидеть роскошную отделку с использованием дорогих материалов, таких как кожа и дерево. В то же время, автомобиль оснащен различными технологиями для обеспечения безопасности, включая системы видеонаблюдения и защиты от нападений.

3. **Технические характеристики**: Под капотом у "Джихад-Мобиля" скрывается мощный двигатель, к